# Opérations de mis-à-jour

In [1]:
from pymongo import MongoClient

def connect_to_db(url=None):
    return MongoClient(url)

connection = connect_to_db("localhost:27017")
db = connection["update_operation"]

## $set

Modifie un dictionnaire avec la valeur associé.   
Si certains champs n'existe pas ils seront crées.  

In [ ]:
collection = db["set"]
collection.insert_one({"id": 30, "dessert": "Glace à la vanille"})

print("Notre glace :", collection.find().to_list())

# on change la description du dessert pour une mousse
collection.update_one({"id": 30}, {"$set": {"dessert": "Mousse au chocolat"}})
print("Maintenant, mousse au chocolat : ", collection.find().to_list())

# $set va aussi créer les champs si besoin
collection.update_one(
    {"id": 30}, {"$set": {"accompagnement": "Copeaux de chocolat blanc"}}
)
print("Se transforme en un nouveau dessert :", collection.find().to_list())

Notre glace : [{'_id': ObjectId('688773a03d4b6070c15657c9'), 'id': 30, 'dessert': 'Glace à la vanille'}, {'_id': ObjectId('688773d73d4b6070c15657ca'), 'id': 30, 'dessert': 'Glace à la vanille'}]
est devenu Mousse au chocolat [{'_id': ObjectId('688773a03d4b6070c15657c9'), 'id': 30, 'dessert': 'Mousse au chocolat'}, {'_id': ObjectId('688773d73d4b6070c15657ca'), 'id': 30, 'dessert': 'Glace à la vanille'}]
Notre nouveau dessert : [{'_id': ObjectId('688773a03d4b6070c15657c9'), 'id': 30, 'dessert': 'Mousse au chocolat', 'accompagnement': 'Copeau de chocolat blanc'}, {'_id': ObjectId('688773d73d4b6070c15657ca'), 'id': 30, 'dessert': 'Glace à la vanille'}]


## $unset
Supprime un champs s'il existe.

In [ ]:
collection = db["unset"]
collection.insert_one({"personnage": "superman", "accessoire": "Cape rouge"})

# on supprime le champs 'accessoire'
collection.update_one({"personnage": "superman"}, {"$unset": {"accessoire": ""}})
print("Oh non, Superman à perdu ça cape :", collection.find().to_list())

Oh non, Superman à perdu ça cape : [{'_id': ObjectId('688776473d4b6070c15657cb'), 'personnage': 'superman'}, {'_id': ObjectId('688776723d4b6070c15657cc'), 'personnage': 'superman', 'accessoire': 'Cape rouge'}]


## $inc
Incremente un champ avec la valeur spécifié.  
Si un champs n'existe pas, on le créer en lui assignement la valeur spécifié.  
C'est une opération atomique


In [ ]:
collection = db['inc']
collection.insert_one({'id': 0, 'cookies': 10, 'Coca': 3, 'Pepsi': 1})

collection.update_one({'id': 0}, {'$inc': {'cookies': -2, 'Coca': -1}})
print("Des clients sont passés :", collection.find().to_list())

collection.update_one({'id': 0}, {'$inc': {'cookies': 2, 'Coca': 8, 'Pepsi': 9, 'Croissant': 10}})
print("On a reçu une livraison :", collection.find().to_list())

Des clients sont passés : [{'_id': ObjectId('688779383d4b6070c15657cd'), 'id': 0, 'cookies': 86, 'Coca': 8, 'Pepsi': 10, 'Croissant': 10}, {'_id': ObjectId('68877bcb3d4b6070c15657ce'), 'id': 0, 'cookies': 10, 'Coca': 3, 'Pepsi': 1}, {'_id': ObjectId('68877be23d4b6070c15657cf'), 'id': 0, 'cookies': 10, 'Coca': 3, 'Pepsi': 1}]
On a reçu une livraison : [{'_id': ObjectId('688779383d4b6070c15657cd'), 'id': 0, 'cookies': 88, 'Coca': 16, 'Pepsi': 19, 'Croissant': 20}, {'_id': ObjectId('68877bcb3d4b6070c15657ce'), 'id': 0, 'cookies': 10, 'Coca': 3, 'Pepsi': 1}, {'_id': ObjectId('68877be23d4b6070c15657cf'), 'id': 0, 'cookies': 10, 'Coca': 3, 'Pepsi': 1}]


## $mul

Meme chose que inc mais avec une multiplication au lieu d'une addition.  
Si le champs n'existe pas, il est crée et sa valeur est 0 (on conserve le type).  
C'est une opération atomique
  
/!\ lors de multiplications entre différents types. Des conversions automatiques peuvent etre réalisés. Se référer à la documentation [mongodb](https://www.mongodb.com/docs/manual/reference/operator/update/mul/)

In [ ]:
# on reprend intentionellement la collection de $inc
collection.update_one({'id': 0}, {'$mul': {'cookies': 3}})
print('On a commandé 3 fois trop de cookies. Mince alors !', collection.find().to_list())

On a commandé 3 fois trop de cookies. Mince alors ! [{'_id': ObjectId('688779383d4b6070c15657cd'), 'id': 0, 'cookies': 90, 'Coca': 10, 'Pepsi': 10, 'Croissant': 10}]


## $rename

Renomme un Champs

Opération non atomique (Delete + $set)

In [ ]:
collection = db["rename"]

collection.insert_one({"id": 2, "ortaugrafe": False})
print("Nooon, des fautes d'orthographes :", collection.find().to_list())

collection.update_one({"id": 2}, {"$rename": {"ortaugrafe": "orthographe"}})
print("C'est bon, on est sauf :", collection.find().to_list())


Nooon, des fautes d'orthographes [{'_id': ObjectId('688784333d4b6070c15657d0'), 'id': 2, 'ortaugrafe': False}]
C'est bon, on est sauf [{'_id': ObjectId('688784333d4b6070c15657d0'), 'id': 2, 'orthographe': False}]


## $push
Ajoute un élément à un Array.
Si le champs n'existe pas. Il est crée avec l'élément à l'intérieur de l'array.
Echoue si le champs n'est pas un Array.


In [ ]:
collection = db["push"]
collection.insert_one({'id': 3, 'tableaux': ['Le Radeau de la Méduse', 'Knight at the Crossroads']})
print("Ma collection :", collection.find().to_list())

collection.update_one({'id': 3}, {'$push': {'tableaux': "L'Île des morts"}})
print("Une nouvelle pièce dans ma collection :", collection.find().to_list())

collection.update_one({'id': 3}, {'$push': {'gravures': 'Lucifer, King of Hell'}})
print("Ah, maintenant je sais ca que je regarde !", collection.find().to_list())

Ma collection : [{'_id': ObjectId('688789bd3d4b6070c15657d1'), 'id': 3, 'tableaux': ['Le Radeau de la Méduse', 'Knight at the Crossroads']}]
Une nouvelle pièce dans ma collection : [{'_id': ObjectId('688789bd3d4b6070c15657d1'), 'id': 3, 'tableaux': ['Le Radeau de la Méduse', 'Knight at the Crossroads', "L'Île des morts"]}]
Ah, maintenant je sais ca que je regarde ! [{'_id': ObjectId('688789bd3d4b6070c15657d1'), 'id': 3, 'tableaux': ['Le Radeau de la Méduse', 'Knight at the Crossroads', "L'Île des morts"], 'courants': [['Romantique', 'Symbolisme']]}]


## $pull
Retire d'un Array tous les instances de la valeur spécifié.


In [ ]:
db.close()